# RNN Model

#### Goal of This File:

##### 1. Import Libraries and Helper Functions ==> Data Sourcing and Munging ==> Utility Functions ==> Loading the Data ==> Merging all Files

##### 2. Imbalanced Data Solution ==> Glove Embedding

##### 3. Simple RNN

##### 4. Conclusion ==> Results ==> Tensorboard

## 1. Import Libraries, Helper Functions and Load Constants

In [1]:
!pip install pandas numpy nltk scikit-learn wordcloud seaborn gensim tensorflow imblearn xgboost matplotlib patool unrar > /dev/null

The system cannot find the path specified.


In [2]:
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already u

True

In [3]:
TF_ENABLE_ONEDNN_OPTS = 0
URLS = CHUNKS_URLS
GLOVE_URL = 'https://f005.backblazeb2.com/file/glove-embeddings-dims/glove.6B.100d.txt'
READY_DATA_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/ready_for_models.rar'
CLASS_WEIGHTS_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/class_weights.pickle'
LIMIT = 60
LARGER_DATASET_PATH = "../larger_dataset"
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + "/preprocessed_data_chunks/"
BASE_FILE_NAME = "chunk_"
FILE_FORMAT = ".csv"
ACTUAL_DATA_SHAPE = 6990280
RANDOM_STATE = 42
USE_READY_DATA = True

### 1.1 Data Sourcing and Munging

#### 1.1.1 Loading The Data

In [10]:
if not USE_READY_DATA:
    if not os.path.exists(LARGER_DATASET_PATH):
        os.mkdir(LARGER_DATASET_PATH)
    if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
        os.mkdir(PREPROCESSED_CHUNKS_PATH)
    get_chunks(URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

chunk_1 already exists.
chunk_2 already exists.
chunk_3 already exists.
chunk_4 already exists.
chunk_5 already exists.
chunk_6 already exists.
chunk_7 already exists.
chunk_8 already exists.
chunk_9 already exists.
chunk_10 already exists.
chunk_11 already exists.
chunk_12 already exists.
chunk_13 already exists.
chunk_14 already exists.
chunk_15 already exists.
chunk_16 already exists.
chunk_17 already exists.
chunk_18 already exists.
chunk_19 already exists.
chunk_20 already exists.
chunk_21 already exists.
chunk_22 already exists.
chunk_23 already exists.
chunk_24 already exists.
chunk_25 already exists.
chunk_26 already exists.
chunk_27 already exists.
chunk_28 already exists.
chunk_29 already exists.
chunk_30 already exists.
chunk_31 already exists.
chunk_32 already exists.
chunk_33 already exists.
chunk_34 already exists.
chunk_35 already exists.
chunk_36 already exists.
chunk_37 already exists.
chunk_38 already exists.
chunk_39 already exists.
chunk_40 already exists.
chunk_41 

#### 1.1.2 Merging all Individual Files

In [11]:
#get all names of downloaded files
if not USE_READY_DATA:
    all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [12]:
#read all chunks into a list
if not USE_READY_DATA:
    list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [13]:
#concatenate all chunks into a singular df
if not USE_READY_DATA:
    df = group_up_chunks(list_dfs)

In [14]:
#check how much of the data was actually downloaded
if not USE_READY_DATA:
    percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
    percent_loaded

100.0

In [16]:
if not USE_READY_DATA:
    review_data = df[['text', 'stars']].copy()

In [18]:
if not USE_READY_DATA:
    review_data.reset_index(inplace = True)
    review_data.drop(['index'],axis = 1,inplace = True)

In [19]:
if not USE_READY_DATA:
    review_data.rename(columns = {'text':'full_review_text','stars':'star_rating'}, inplace = True)

In [21]:
if not USE_READY_DATA:
    review_data.isnull().sum()

full_review_text    44
star_rating          0
dtype: int64

In [22]:
if not USE_READY_DATA:
    review_data.dropna()

,full_review_text,star_rating
0,decide eat aware go take 2 hour begin end try ...,3.0
1,ve take lot spin class year nothing compare cl...,5.0
2,family diner buffet eclectic assortment large ...,3.0
3,wow yummy different delicious favorite lamb cu...,5.0
4,cute interior owner give u tour upcoming patio...,4.0
...,...,...
6990275,late addition service iccu apple pay iccu debi...,5.0
6990276,spot offer great affordable east weekend paddl...,5.0
6990277,home depot need get lot demential lumber seem ...,4.0
6990278,m feel like ignore caloriecounting indulge fla...,5.0


## 2. Imbalanced Data Solution

In [23]:
if not USE_READY_DATA:
    X = review_data['full_review_text']
    y = review_data['star_rating']

In [25]:
if not USE_READY_DATA:
    X = pd.Series([str(text) for text in X])

In [26]:
if not USE_READY_DATA:
    y = [label - 1 for label in y]

### 2.1 Class Weights

In [27]:
if not os.path.exists("../pickle_files"):
    os.mkdir("../pickle_files")

In [28]:
class_weights = compute_class_weight('balanced', classes=[0,1,2,3,4], y=y)

In [29]:
if not os.path.exists("../pickle_files/class_weights.pickle"):
    pickle_out = open("../pickle_files/class_weights.pickle",'wb')
    pickle.dump(class_weights,pickle_out)
    pickle_out.close()

### 2.2 GloVe + Hyperparamters

In [28]:
if not os.path.exists('../glove_files'):
    os.mkdir('../glove_files')

In [29]:
if not os.path.exists('../glove_files/glove.6B.100d.txt'):
    print("Downloading GloVe File...")
    r = requests.get(GLOVE_URL)
    with open("../glove_files/glove.6B.100d.txt", 'wb') as fd:
        for data in r.iter_content():
            #save file in the current directory of the notebook
            fd.write(data)

In [30]:
UNIQUE_WORDS = set(' '.join(X).split())

##### Hyperparameters

In [31]:
VOCAB_SIZE = len(UNIQUE_WORDS)
EMBEDDING_DIM = 100
MAX_TEXT_LEN = 200
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOKEN = "<OOV>"
BATCH_SIZE = 64
EPOCHS = 15

##### ==> here is a quick explaination of how the dataset will be split using a smaller sample example.
##### ==> dataset => 100
##### ==> train_set => tr_s (example: 80)
##### ==> valid_set => vs (example: 10)
##### ==> test_set => te_s (example: 10)
##### ==> t = tr_s (80) + vs (10)
##### ==> train_set = x[:80]
##### ==> valid_set = x[80:t]
##### ==> test_set = x[t:] why t? because => vs = ts

##### use this guideline if you are confused about how the train-validation-test split was done. Also, this is a future guide for me as well in case I forget.

* train_set_size = 6,990,280 * 0.8 = 5,592,224
* valid_set_size = 6,990,280 * 0.1 = 699,028
* train_plus_valid = 5,592,224 + 699,028 = 6,291,252

==> To Confirm: test_size = 6,990,280 - 6,291,252 = 699,028

* train_set = [0:5,592,224]
* train_labels = [0:5,592,224]
* validation_set = [5,592,224:6,291,252] ==> 699,028
* validation_labels = [5,592,224:6,291,252] ==> 699,028
* test_set = [6,291,252,6,990,280] ==> 699,028
* test_labels = [6,291,252,6,990,280] ==> 699,028

In [32]:
TRAIN_PERCENT = 0.8
VALID_TEST_PERCENT = 0.1
TRAIN_SIZE = int(ACTUAL_DATA_SHAPE * TRAIN_PERCENT)
VALID_TEST_SIZE = int(ACTUAL_DATA_SHAPE * VALID_TEST_PERCENT)
TOTAL_TEST_SIZE = TRAIN_SIZE + VALID_TEST_SIZE
train_set = X[:TRAIN_SIZE]
train_labels = y[:TRAIN_SIZE]
validation_set = X[TRAIN_SIZE:TOTAL_TEST_SIZE]
validation_labels = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set = X[TOTAL_TEST_SIZE:]
test_labels = y[TOTAL_TEST_SIZE:]

In [ ]:
READY_DATASET_PATH = "../larger_dataset/ready_for_models/"

In [ ]:
if os.path.exists("../larger_dataset"):
    os.mkdir("../larger_dataset")
    os.mkdir(READY_DATASET_PATH)

In [ ]:
get_chunks(READY_DATA_URL,0,1,'ready_for_models','../larger_dataset/ready_for_models/','.rar',False)
patoolib.extract_archive(READY_DATASET_PATH + "ready_for_models.rar", outdir = READY_DATASET_PATH)

In [ ]:
train_set_padded = pickle.load(open(READY_DATASET_PATH + "train_set_padded.pickle", "rb"))
train_labels = pickle.load(open(READY_DATASET_PATH + "train_labels.pickle", "rb"))
valid_set_padded = pickle.load(open(READY_DATASET_PATH + "valid_set_padded.pickle", "rb"))
validation_labels = pickle.load(open(READY_DATASET_PATH + "validation_labels.pickle", "rb"))
test_set_padded = pickle.load(open(READY_DATASET_PATH + "test_set_padded.pickle", "rb"))
test_labels = pickle.load(open(READY_DATASET_PATH + "test_labels.pickle", "rb"))
EMBEDDING_LAYER = pickle.load(open(READY_DATASET_PATH + "EMBEDDING_LAYER.pickle", "rb"))

In [34]:
# NOTE: THIS CELL TAKES A WHILE TO RUN. USE THE PICKLE FILES IMPORTED FROM THE ABOVE CELLS.
if not os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
    tokenizer.fit_on_texts(train_set)
    words_to_index = tokenizer.word_index

In [35]:
# NOTE: THIS CELL TAKES A WHILE TO RUN. USE THE PICKLE FILES IMPORTED FROM THE ABOVE CELLS.
if not os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    train_sequences = tokenizer.texts_to_sequences(train_set)
    train_set_padded = pad_sequences(train_sequences, maxlen=MAX_TEXT_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)
    
if not os.path.exists(READY_DATASET_PATH + "valid_set_padded.pickle"):
    valid_sequences = tokenizer.texts_to_sequences(validation_set)
    valid_set_padded = pad_sequences(valid_sequences, maxlen=MAX_TEXT_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)
    
if not os.path.exists(READY_DATASET_PATH + "test_set_padded.pickle"):
    test_sequences = tokenizer.texts_to_sequences(test_set)
    test_set_padded = pad_sequences(test_sequences, maxlen=MAX_TEXT_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)

In [37]:
if not os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    train_set_padded = np.array(train_set_padded)
    
if not os.path.exists(READY_DATASET_PATH + "train_labels.pickle"):
    train_labels = np.array(train_labels)
    
if not os.path.exists(READY_DATASET_PATH + "valid_set_padded.pickle"):   
    valid_set_padded = np.array(valid_set_padded)

if not os.path.exists(READY_DATASET_PATH + "validation_labels.pickle"):
    validation_labels = np.array(validation_labels)

if not os.path.exists(READY_DATASET_PATH + "test_set_padded.pickle"):
    test_set_padded = np.array(test_set_padded)

if not os.path.exists(READY_DATASET_PATH + "test_labels.pickle"):
    test_labels = np.array(test_labels)

In [38]:
word_to_vec_map = read_glove_vector('../glove_files/glove.6B.100d.txt')

In [39]:
if not os.path.exists(READY_DATASET_PATH + "EMBEDDING_LAYER.pickle"):
    vocab_mapping = len(words_to_index)
    embed_vector_len = word_to_vec_map['moon'].shape[0]

    emb_matrix = np.zeros((vocab_mapping, embed_vector_len))

    for word, index in words_to_index.items():
        embedding_vector = word_to_vec_map.get(word)
        if embedding_vector is not None:
            emb_matrix[index, :] = embedding_vector

    EMBEDDING_LAYER = Embedding(input_dim=vocab_mapping,\
                                output_dim=embed_vector_len, input_length=MAX_TEXT_LEN, weights = [emb_matrix], trainable=False)

In [41]:
if os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    print("Found Pickle File!")
    train_set_padded = pickle.load(open(READY_DATASET_PATH + "train_set_padded.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "train_set_padded.pickle",'wb')
    pickle.dump(train_set_padded,pickle_out)
    pickle_out.close()
    
if os.path.exists(READY_DATASET_PATH + "train_labels.pickle"):
    print("Found Pickle File!")
    train_labels = pickle.load(open(READY_DATASET_PATH + "train_labels.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "train_labels.pickle",'wb')
    pickle.dump(train_labels,pickle_out)
    pickle_out.close()

In [42]:
if os.path.exists(READY_DATASET_PATH + "valid_set_padded.pickle"):
    print("Found Pickle File!")
    valid_set_padded = pickle.load(open(READY_DATASET_PATH + "valid_set_padded.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "valid_set_padded.pickle",'wb')
    pickle.dump(valid_set_padded,pickle_out)
    pickle_out.close()
    
if os.path.exists(READY_DATASET_PATH + "validation_labels.pickle"):
    print("Found Pickle File!")
    validation_labels = pickle.load(open(READY_DATASET_PATH + "validation_labels.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "validation_labels.pickle",'wb')
    pickle.dump(validation_labels,pickle_out)
    pickle_out.close()

In [43]:
if os.path.exists(READY_DATASET_PATH + "test_set_padded.pickle"):
    print("Found Pickle File!")
    test_set_padded = pickle.load(open(READY_DATASET_PATH + "test_set_padded.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "test_set_padded.pickle",'wb')
    pickle.dump(test_set_padded,pickle_out)
    pickle_out.close()
    
if os.path.exists(READY_DATASET_PATH + "test_labels.pickle"):
    print("Found Pickle File!")
    test_labels = pickle.load(open(READY_DATASET_PATH + "test_labels.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "test_labels.pickle",'wb')
    pickle.dump(test_labels,pickle_out)
    pickle_out.close()

In [44]:
if os.path.exists(READY_DATASET_PATH + "EMBEDDING_LAYER"):
    print("Found Pickle File!")
    EMBEDDING_LAYER = pickle.load(open(READY_DATASET_PATH + "EMBEDDING_LAYER.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "EMBEDDING_LAYER.pickle",'wb')
    pickle.dump(EMBEDDING_LAYER,pickle_out)
    pickle_out.close()

## 3. RNN

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [45]:
tf.keras.backend.clear_session()
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
rnn_model = tf.keras.models.Sequential([
    EMBEDDING_LAYER,
    tf.keras.layers.SimpleRNN(60),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(5, activation = "softmax")
])

2023-05-06 06:25:42.794625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-06 06:25:42.805176: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-06 06:25:42.805713: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [46]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          100208800 
                                                                 
 simple_rnn (SimpleRNN)      (None, 60)                9660      
                                                                 
 dense (Dense)               (None, 512)               31232     
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 100,252,257
Trainable params: 43,457
Non-trainable params: 100,208,800
_________________________________________________________________


In [47]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [48]:
rnn_model.compile(optimizer=Adam(), loss = SparseCategoricalCrossentropy(), metrics=['accuracy',custom_f1_score])
rnn_model.fit(train_set_padded, train_labels,validation_data = (valid_set_padded,validation_labels),\
              batch_size=BATCH_SIZE, epochs=EPOCHS,class_weight=dict(enumerate(class_weights)),\
              callbacks=[tensorboard_callback,EarlyStopping(patience=3),ReduceLROnPlateau(factor=0.1, patience=1)])

2023-05-06 06:25:44.419360: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4473779200 exceeds 10% of free system memory.


Epoch 1/15


2023-05-06 06:25:47.395668: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-06 06:25:47.431947: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f89b000caa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-06 06:25:47.431980: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-05-06 06:25:47.438291: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-06 06:25:47.554077: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-06 06:25:47.662562: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

21004/87379 [======>.......................] - ETA: 1:16:47 - loss: 1.6073 - accuracy: 0.2519 - custom_f1_score: 7.7324

KeyboardInterrupt: 

In [ ]:
if not os.path.exists("../saved_models"):
    os.mkdir("../saved_models")

In [ ]:
rnn_model.save_weights('../saved_models/rnn_model.h5')

## 4. Conclusion

### 4.1 Results

### 4.2 Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit